In [1]:
# keras CNN predict air, between, ground or not jumping

In [2]:
!pip3 install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pickle

from my_classes import DataGeneratorSkillBorders

2024-06-10 23:25:02.175435: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-10 23:25:02.215609: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 23:25:03.018995: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
video_border_labels_path = 'data/df_video_border_labels.pkl'

train_videos = [
    'videos/20240201_atelier_001.mp4',
    'videos/20240201_atelier_002.mp4',
    'videos/20240201_atelier_003.mp4',
    'videos/20240201_atelier_004.mp4',
    'videos/20240201_atelier_005.mp4',
    'videos/20240209_atelier_006.mp4',
    'videos/20240209_atelier_007.mp4',
    'videos/20240209_atelier_008.mp4',
]

In [5]:
def pickle_load_or_create(path, cols):
    if os.path.exists(path):
        with open(path, 'rb') as file:
            return pickle.load(file)
    else:
        return pd.DataFrame(columns=cols)

In [6]:
df_labels = pickle_load_or_create(video_border_labels_path, [])
df_labels

,path,frame,border
0,videos/20240209_atelier_006.mp4,745,0
1,videos/20240209_atelier_007.mp4,103,0
2,videos/20240201_atelier_002.mp4,91,0
3,videos/20240201_atelier_004.mp4,806,2
4,videos/20240201_atelier_003.mp4,622,2
...,...,...,...
653,videos/20240201_atelier_003.mp4,247,2
654,videos/20240201_atelier_002.mp4,370,2
655,videos/20240201_atelier_003.mp4,242,1
656,videos/20240201_atelier_005.mp4,2390,9


In [7]:
df_labels.loc[df_labels.border == 5, 'border'] = 3
df_labels.loc[df_labels.border == 9, 'border'] = 4

In [8]:
def get_random_frame(videos, grey=True, scale=1):
    """
    videos: array of video_paths
    df_video_border_labels: panda dataframe ['path', 'frame', 'borderlabel']
        [ 0 : ground
          1 : heels of ground
          2 : air ]
    """
    path = videos[np.random.randint(0, len(videos)-1)]
    
    cap = cv2.VideoCapture(path)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_nr = np.random.randint(0, video_length-1)
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_nr)
    res, frame = cap.read()
    frame = cv2.resize(frame, dsize=(0,0), fx=scale, fy=scale)
    # frame = cv2.cvtColor(frame, 7)
    cap.release()
    # cv2.destroyAllWindows()

    return path, frame_nr, frame

In [9]:
input_shape = get_random_frame(train_videos, grey=True, scale=0.4)[2].shape
input_shape

(768, 432, 3)

In [10]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64,64,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


/home/miked/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,320 (220.00 KB)

 Trainable params: 56,320 (220.00 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
unique_labels = df_labels['border'].unique()
unique_labels

array([0, 2, 1, 4, 3])

In [13]:
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(5))

In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │       147,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            85 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 203,877 (796.39 KB)

 Trainable params: 203,877 (796.39 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
from my_classes import DataGeneratorSkillBorders
# Parameters
params = {'dim': (64,64),
          'batch_size': 16,
          'n_classes': len(unique_labels),
          'n_channels': 3,
          'shuffle': True,
          'train': True}

training_generator = DataGeneratorSkillBorders(df_labels, **params)
test_generator = DataGeneratorSkillBorders(df_labels, train=False)

                                path  frame  border
74   videos/20240201_atelier_001.mp4    446       0
464  videos/20240201_atelier_002.mp4    916       0
29   videos/20240209_atelier_007.mp4    293       2
304  videos/20240209_atelier_007.mp4    778       0
320  videos/20240201_atelier_005.mp4    479       1
..                               ...    ...     ...
375  videos/20240209_atelier_007.mp4    666       0
71   videos/20240209_atelier_006.mp4    854       4
582  videos/20240209_atelier_007.mp4    491       2
615  videos/20240201_atelier_003.mp4    312       0
456  videos/20240201_atelier_004.mp4    627       1

[526 rows x 3 columns]
                                path  frame  border
270  videos/20240201_atelier_003.mp4    277       2
255  videos/20240209_atelier_007.mp4    763       0
511  videos/20240201_atelier_004.mp4   2039       4
304  videos/20240209_atelier_007.mp4    778       0
172  videos/20240201_atelier_004.mp4   1398       0
..                               ...    

In [16]:
%%time
X, y = training_generator.__getitem__(3)

CPU times: user 13.5 s, sys: 261 ms, total: 13.8 s
Wall time: 3.17 s


In [17]:
X

array([[[[ 91.,  98., 104.],
         [ 91.,  97., 106.],
         [ 91.,  98., 109.],
         ...,
         [ 79.,  86.,  92.],
         [ 85.,  90.,  96.],
         [ 88.,  95., 101.]],

        [[ 91.,  99., 108.],
         [ 96., 102., 111.],
         [ 92., 100., 107.],
         ...,
         [ 81.,  87.,  91.],
         [ 85.,  91.,  95.],
         [ 90.,  97.,  98.]],

        [[ 89.,  95., 101.],
         [ 92., 100., 109.],
         [ 96., 104., 113.],
         ...,
         [ 77.,  84.,  90.],
         [ 79.,  85.,  89.],
         [ 88.,  92.,  96.]],

        ...,

        [[154., 152., 151.],
         [155., 155., 153.],
         [159., 156., 155.],
         ...,
         [ 19.,  33.,  47.],
         [ 21.,  34.,  50.],
         [ 22.,  34.,  53.]],

        [[150., 150., 148.],
         [148., 148., 148.],
         [150., 150., 150.],
         ...,
         [ 19.,  33.,  52.],
         [ 17.,  32.,  48.],
         [ 16.,  28.,  47.]],

        [[138., 138., 138.],
       

In [18]:
y

array([0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 4, 2])

In [19]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(training_generator, epochs=10, 
                    validation_data=training_generator)

Epoch 1/10


2024-06-10 23:25:26.424855: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 3 of 8
2024-06-10 23:25:39.804960: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 6 of 8


 1/32 ━━━━━━━━━━━━━━━━━━━━ 17:41 34s/step - accuracy: 0.3125 - loss: 16.5645

2024-06-10 23:25:48.593686: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


32/32 ━━━━━━━━━━━━━━━━━━━━ 278s 8s/step - accuracy: 0.3883 - loss: 6.8093 - val_accuracy: 0.4746 - val_loss: 1.3129
Epoch 2/10


...

32/32 ━━━━━━━━━━━━━━━━━━━━ 316s 8s/step - accuracy: 0.6093 - loss: 1.0127 - val_accuracy: 0.6660 - val_loss: 0.9331
Epoch 10/10


2024-06-11 00:07:15.921691: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 3 of 8
2024-06-11 00:07:33.399465: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 7 of 8


 1/32 ━━━━━━━━━━━━━━━━━━━━ 18:10 35s/step - accuracy: 0.6875 - loss: 1.0681

2024-06-11 00:07:37.976500: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


32/32 ━━━━━━━━━━━━━━━━━━━━ 275s 8s/step - accuracy: 0.6578 - loss: 0.9659 - val_accuracy: 0.6543 - val_loss: 0.8989


In [20]:
model.fit?

Signature:
model.fit(
    x=None,
    y=None,
    batch_size=None,
    epochs=1,
    verbose='auto',
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
)
Docstring:
Trains the model for a fixed number of epochs (dataset iterations).

Args:
    x: Input data. It could be:
        - A NumPy array (or array-like), or a list of arrays
        (in case the model has multiple inputs).
        - A tensor, or a list of tensors
        (in case the model has multiple inputs).
        - A dict mapping input names to the corresponding array/tensors,
        if the model has named inputs.
        - A `tf.data.Dataset`. Should return a tuple
        of either `(inputs, targets)` or
        `(inputs, targets, sample_weights)`.
        - A `keras.utils.PyDataset` returning `(inputs,
      

In [21]:
df_labels = pickle_load_or_create(video_border_labels_path, [])
df_labels

,path,frame,border
0,videos/20240209_atelier_006.mp4,745,0
1,videos/20240209_atelier_007.mp4,103,0
2,videos/20240201_atelier_002.mp4,91,0
3,videos/20240201_atelier_004.mp4,806,2
4,videos/20240201_atelier_003.mp4,622,2
...,...,...,...
653,videos/20240201_atelier_003.mp4,247,2
654,videos/20240201_atelier_002.mp4,370,2
655,videos/20240201_atelier_003.mp4,242,1
656,videos/20240201_atelier_005.mp4,2390,9


In [22]:
df_labels = pickle_load_or_create(video_border_labels_path, [])
unique_labels = df_labels['border'].unique()
unique_labels

array([0, 2, 1, 9, 5])